In [61]:
from django.shortcuts import render
from django.db.models.functions import TruncDate
from django.db.models import Count
from django.contrib.auth.decorators import login_required
from django.utils import timezone

import pandas as pd

from repository.models import Commit
from repository.utils import date_range
import numpy as np

In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [98]:
commits = Commit.objects.all()

now = timezone.now()
start = now - timezone.timedelta(days=363)
commits = commits.filter(created__range=[start, now])

# Truncate datetime to date
commits = commits.annotate(date=TruncDate('created'))

count = commits.values('date').annotate(count=Count('date')).order_by('date')

df = pd.DataFrame(list(count))
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)

idx = pd.date_range(start=start, end=now).date
df = df.reindex(idx, fill_value=0)

df

,count
date,
2021-05-15,0
2021-05-16,9
2021-05-17,0
2021-05-18,0
2021-05-19,3
...,...
2022-05-09,0
2022-05-10,10
2022-05-11,1


In [101]:
weekdays = [[] for _ in range(7)]
dates = [[] for _ in range(7)]

for date, count in df.iterrows():
    i = date.weekday()
    weekdays[i].append(int(count))
    dates[i].append(date)

print(weekdays)

[[0, 4, 0, 9, 0, 0, 0, 0, 0, 0, 2, 9, 5, 10, 0, 6, 4, 8, 9, 9, 0, 0, 0, 7, 0, 9, 0, 0, 3, 0, 0, 0, 0, 0, 7, 3, 0, 1, 0, 3, 6, 0, 0, 0, 1, 3, 5, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 0, 10, 0, 8, 0, 0, 0, 7, 3, 6, 9, 0, 0, 9, 0, 4, 0, 0, 0, 6, 0, 0, 8, 8, 0, 5, 3, 10, 6, 0, 0, 9, 0, 2, 0, 4, 0, 3, 5, 8, 0, 0, 2, 3, 2, 2, 10], [3, 0, 0, 0, 0, 0, 0, 0, 6, 4, 3, 0, 9, 0, 8, 6, 0, 0, 10, 9, 0, 0, 0, 0, 9, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 8, 6, 7, 0, 0, 0, 7, 9, 0, 7, 1], [0, 0, 10, 0, 0, 0, 1, 8, 1, 0, 0, 9, 0, 0, 5, 3, 0, 0, 0, 1, 1, 0, 0, 0, 0, 8, 0, 0, 5, 5, 0, 9, 0, 0, 6, 0, 0, 0, 8, 5, 0, 7, 7, 0, 10, 0, 8, 7, 0, 5, 7, 0], [0, 3, 0, 8, 0, 3, 0, 5, 0, 7, 8, 8, 3, 10, 9, 0, 0, 0, 0, 1, 0, 0, 6, 8, 6, 0, 0, 0, 0, 6, 8, 7, 0, 0, 0, 0, 0, 5, 0, 8, 8, 5, 5, 0, 0, 0, 0, 1, 0, 9, 0, 2], [0, 0, 0, 1, 0, 9, 6, 0, 0, 0, 2, 1, 0, 5, 0, 0, 0, 5, 4, 6, 0, 0, 0, 1, 4, 3, 8, 6, 8, 0, 10, 0, 8, 8, 5, 0, 2, 0, 0, 3, 2, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 0], [9, 8, 0, 0, 0, 10, 0, 0, 10, 0, 0, 0, 0, 

52

In [100]:
import plotly.express as px

# a = [l[:52] for l in weekdays]

fig = px.imshow(weekdays)

fig.show()